# Fluid dynamics · Taylor–Green walkthrough

This notebook mirrors the viscosity recovery test in `src/fluid_dynamics/taylor_green_benchmark.py`. We initialise the lattice with a Taylor–Green vortex, step a short run, and monitor the decaying kinetic energy.


In [ ]:
import sys
from pathlib import Path


def _ensure_repo_on_path():
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for candidate in candidates:
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            sys.path.insert(0, str(candidate))
            return candidate
    return cwd

_ensure_repo_on_path()

from notebooks.utils import ensure_repo_path, allocate_artifacts, preview_json

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")



In [ ]:
try:
    from src.fluid_dynamics.fluids.lbm2d import LBMConfig, LBM2D
    from src.fluid_dynamics.taylor_green_benchmark import init_taylor_green
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from src.fluid_dynamics.fluids.lbm2d import LBMConfig, LBM2D
    from src.fluid_dynamics.taylor_green_benchmark import init_taylor_green

import numpy as np

cfg = LBMConfig(nx=64, ny=64, tau=0.9, periodic_x=True, periodic_y=True, void_enabled=False, u_clamp=0.12)
sim = LBM2D(cfg)
init_taylor_green(sim, U0=0.05, k=2*np.pi)

energies = []
for step in range(120):
    sim.step(1)
    if step % 10 == 0:
        sim.moments()
        E = 0.5 * float(np.mean(sim.ux**2 + sim.uy**2))
        energies.append(E)
        print(f"step {step:03d}: E={E:.6f}")

if energies:
    decay_ratio = energies[-1] / energies[0]
    print(preview_json({"samples": len(energies), "final_over_initial": decay_ratio}))


In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image, display

artifacts = allocate_artifacts("fluid_dynamics", "taylor_green_notebook_demo")
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(np.arange(0, len(energies)) * 10, energies, marker='o', label='E(t) samples')
ax.set_xlabel('Step')
ax.set_ylabel('E(t)')
ax.set_title('Taylor–Green energy decay (notebook demo)')
ax.legend()
fig.tight_layout()
fig.savefig(artifacts['figure'], dpi=160)
plt.close(fig)
print(preview_json({name: str(path) for name, path in artifacts.items()}))
display(Image(filename=str(artifacts['figure'])))


## Notebook checklist

* `init_taylor_green` seeds the velocity field; the reduced step count keeps the demo snappy.
* Monitor the energy ratio to verify the expected exponential decay before running long benchmarks.
* Artifact paths align with the CLI so notebook figures/logs coexist with scripted runs.
* Increase `steps`/`sample_every` when you need precise viscosity fits.
